In [ ]:
import ipyparallel as ipp

cluster = ipp.Client(profile='ssh')

cluster.ids

dview = cluster[:]
dv = cluster[:]

view = cluster.load_balanced_view()

In [ ]:
def test(a,b):
    return a+b

In [ ]:
%%timeit
async_results = []

for i in range(10):
    for j in range(10):
        # This line submits the tasks for parallel computation.
        ar = view.apply_async(test, i,j)
        async_results.append(ar)

cluster.wait_interactive(async_results) 

In [ ]:
%%timeit
async_results = []

for i in range(10):
    for j in range(10):
        # This line submits the tasks for parallel computation.
        ar = test(i,j)
        async_results.append(ar)


In [ ]:
%timeit dv.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [ ]:
%timeit cluster[0].map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

In [ ]:
%time
serial_result = map(lambda x:x**10, range(32))

In [ ]:
%time
parallel_result = dview.map_sync(lambda x: x**10, range(32))

In [ ]:
serial_result==parallel_result

In [ ]:
lbv = ipclient.load_balanced_view()

In [ ]:
len(lbv)

In [ ]:
ipclient.profile

In [ ]:
e = cluster.become_dask()  # start dask on top of IPyParallel
e

In [ ]:
import distributed

In [ ]:
dclient = distributed.worker_client(ipclient)

In [ ]:
# Dask Client.get method computes dask graphs on the cluster.
dclient.get({'a': 41, 'b': (lambda x: x + 1, 'a')}, 'b')

In [ ]:
# Dask Client.get method computes dask graphs on the cluster.
dclient.get({'a': 41, 'b': (lambda x: x + 1, 'a')}, 'b')

In [ ]:
from dask.distributed import Client
from time import sleep
import random

def inc(x):
    sleep(random.random() / 10)
    return x + 1

def dec(x):
    sleep(random.random() / 10)
    return x - 1

def add(x, y):
    sleep(random.random() / 10)
    return x + y


client = Client('192.168.1.4:8786')

In [ ]:
client.restart()

In [ ]:
from dask.diagnostics import ProgressBar

In [ ]:
from os import sys, path
sys.path.append(path.dirname(path.dirname(path.abspath('__file__'))))

import pprint

from sklearn import metrics
from sklearn import preprocessing

import h5py

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.decomposition import PCA

import os
from sklearn.model_selection import cross_val_predict

from sklearn.naive_bayes import GaussianNB
import matplotlib
from functools import reduce

In [ ]:
import dask.dataframe as dd

In [ ]:
from dask.distributed import Client, progress
c = Client()

In [ ]:
final = dd.read_csv('./final.csv')

In [ ]:
final = final.drop('Unnamed: 0', axis=1)

In [ ]:
final.result_final.value_counts().compute()

In [ ]:
# final['over_under_1.5']

In [ ]:
# final.set_index('Unnamed: 0')

In [ ]:
np.random.seed(0)

In [ ]:
pred_data = pd.read_csv('../data/predict.csv', index_col='Unnamed: 0')
df_teams = pred_data[['home_team','away_team']].values

In [ ]:
final = pd.read_csv('./final.csv', index_col='Unnamed: 0')

In [ ]:
final.shape

In [ ]:
final.head()

In [ ]:
ht = final.home_team.values
at = final.away_team.values
all_teams = np.append(ht,at)
un = np.unique(all_teams)
un.shape

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(un)

In [ ]:
# final.columns

In [ ]:
final.loc[:,'home_team'] = le.transform(final.home_team.values)
final.loc[:,'away_team'] =  le.transform(final.away_team.values)

In [ ]:
# del final

In [ ]:
# result_key = 'over_under_2.5' 
result_key = 'result_final' 

In [ ]:
# final2 = final[(final.EH!=100) & (final.EA!=100)].iloc[:]

In [ ]:
def filter_out_100(data):
    print('[Filter]')
    dataf = data[(data.EH!=100) | (data.EA!=100)].iloc[:]
    return dataf

In [ ]:
def plot_data_labels(data, labels=['1','X','2'], npoints=1000):
    x = data.iloc[:npoints]['EH'].values
    y = data.iloc[:npoints]['EA'].values
    label = data.iloc[:npoints][result_key].values
    colors = ['green','blue','black']

    fig = plt.figure(figsize=(8,8))
    plt.scatter(x, y, c=label, cmap=matplotlib.colors.ListedColormap(colors))
    plt.xlabel('EH')
    plt.ylabel('EA')
    cb = plt.colorbar()
    loc = np.arange(0,max(label),max(label)/float(len(colors)))
    cb.set_ticks(loc)
    cb.set_ticklabels(labels)
    return data

In [ ]:
def run_pca(data):
    print('[PCA]')

    pca = PCA(n_components=2)

    # print(pca.explained_variance_ratio_)  

    # print(pca.singular_values_)  

    pca.fit(data.values[:,[2,3]])

    tx =  pca.transform(data.values[:,[2,3]]) 

    data.loc[:,['EH','EA']] = tx
    return data

In [ ]:
def classify(data):
    print('[Classifier]')
    X = data.iloc[:][['home_team','away_team','EH','EA']]#.values
    # X = final.iloc[:][['EH','EA']]#.values
    y = data.iloc[:][result_key]#.values

    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

    # final.iloc[:50000].tail()

    clf = GaussianNB()
    clf = clf.fit(X_train, y_train)

    print('Accuracy: ',clf.score(X_test, y_test))
    # print(metrics.accuracy_score(clf.predict(X_test), y_test))
    # clf.export('tpot_mnist_pipeline.py')

    predictions = clf.predict(X_test)

    pprint.pprint(metrics.classification_report(predictions, y_test))

#     predicted = cross_val_predict(clf, X,y, cv=5)
    return data

In [ ]:
def pipeline_func(data, fns):
    return reduce(lambda a,x: x(a), fns, data)

In [ ]:
# fns = [filter_out_100,plot_data_labels, classify, run_pca, plot_data_labels, classify]
# fns = [filter_out_100,plot_data_labels, run_pca, plot_data_labels]
fns = [filter_out_100, run_pca, classify]

In [ ]:
out = pipeline_func(final,fns)

In [ ]:
data = filter_out_100(final)

In [ ]:
clf1 = GaussianNB()

In [ ]:
X = data.iloc[:][['home_team','away_team','EH','EA']]#.values
# X = final.iloc[:][['EH','EA']]#.values
y = data.iloc[:][result_key]#.values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

# final.iloc[:50000].tail()

# clf = GaussianNB()
clf1 = clf1.fit(X_train, y_train)

print('Accuracy: ',clf1.score(X_test, y_test))
# print(metrics.accuracy_score(clf.predict(X_test), y_test))
# clf.export('tpot_mnist_pipeline.py')

predictions1 = clf1.predict(X_test)
pprint.pprint(metrics.classification_report(predictions1, y_test))


In [ ]:
data1 = run_pca(data)

In [ ]:
final.head()

In [ ]:
data.head()

In [ ]:
X = data1.iloc[:][['home_team','away_team','EH','EA']]#.values
# X = final.iloc[:][['EH','EA']]#.values
y = data1.iloc[:][result_key]#.values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

# final.iloc[:50000].tail()

# clf = GaussianNB()
clf2 = clf2.fit(X_train, y_train)

print('Accuracy: ',clf2.score(X_test, y_test))
# print(metrics.accuracy_score(clf.predict(X_test), y_test))
# clf.export('tpot_mnist_pipeline.py')

predictions2 = clf2.predict(X_test)
pprint.pprint(metrics.classification_report(predictions2, y_test))

In [ ]:
def choose(p1,p2, key=[0,0]):
    if p1==key[0]:
        return p1
    elif p2==key[1]:
        return p2
    else:
        return np.nan

In [ ]:
output_df = pd.DataFrame(list(zip(predictions1,predictions2, y_test)), columns=['p1','p2','ytrue'])

out1 = []
for i, (ix, row) in enumerate(output_df.iterrows()):
    out1.append(choose(row.p1,row.p2, key=[0, 0]))

out2 = []
for i, (ix, row) in enumerate(output_df.iterrows()):
    out2.append(choose(row.p1,row.p2,key=[1, 1]))

In [ ]:
output_df.loc[:, 'fil1'] = out1

In [ ]:
output_df.loc[:, 'fil2'] = out2

In [ ]:
fi3 = output_df[['fil1', 'fil2']].sum(axis=1).values.astype(int)

output_df.loc[:,'fil3'] = fi3

In [ ]:
print(metrics.accuracy_score(output_df['fil3'].values.astype(int), output_df['ytrue'].values.astype(int)))

In [ ]:
ytrue = output_df['ytrue'].values.astype(int)
fi = output_df['fil3'].values.astype(int)

In [ ]:
pprint.pprint(metrics.classification_report(fi, ytrue))

In [ ]:
metrics.accuracy_score()

In [ ]:
output

In [ ]:
x = final2['EH'].values
y = final2['EA'].values
label = final2['over_under_2.5'].values
colors = ['green','black']

fig = plt.figure(figsize=(8,8))
plt.scatter(x, y, c=label, cmap=matplotlib.colors.ListedColormap(colors))
plt.xlabel('EH')
plt.ylabel('EA')
cb = plt.colorbar()
loc = np.arange(0,max(label),max(label)/float(len(colors)))
cb.set_ticks(loc)
cb.set_ticklabels(['over','under'])

In [ ]:
final2.head(2)

### NO PCA

In [ ]:
final2 = final[(final.EH!=100) | (final.EA!=100)].iloc[:20000]

final2.shape


X = final2.iloc[:][['home_team','away_team','EH','EA']]#.values
# X = final.iloc[:][['EH','EA']]#.values
y = final2.iloc[:]['over_under_2.5']#.values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

# final.iloc[:50000].tail()

clf = GaussianNB()
clf = clf.fit(X_train, y_train)

print('Accuracy: ',clf.score(X_test, y_test))
# print(metrics.accuracy_score(clf.predict(X_test), y_test))
# clf.export('tpot_mnist_pipeline.py')

predictions = clf.predict(X_test)

pprint.pprint(metrics.classification_report(predictions, y_test))

predicted = cross_val_predict(clf, X,y, cv=5)

In [ ]:
# metrics.precision_score(y, predicted,pos_label=1) 

# with pca

In [ ]:
final2 = final[(final.EH!=100) | (final.EA!=100)].iloc[:20000]

pca.fit(final2.values[:,[2,3]])

tx =  pca.transform(final2.values[:,[2,3]]) 

final2.loc[:,['EH','EA']] = tx

X = final2.iloc[:][['home_team','away_team','EH','EA']]#.values
# X = final.iloc[:][['EH','EA']]#.values
y = final2.iloc[:]['over_under_2.5']#.values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

# final.iloc[:50000].tail()

clf = GaussianNB()

clf = clf.fit(X_train, y_train)

print('Accuracy: ',clf.score(X_test, y_test))

# clf.export('tpot_mnist_pipeline.py')

predictions = clf.predict(X_test)

pprint.pprint(metrics.classification_report(predictions, y_test))

In [ ]:
# X = final.iloc[:50000][['home_team','away_team','EH','EA']]#.values
# # X = final.iloc[:][['EH','EA']]#.values
# y = final.iloc[:50000]['over_under_2.5']#.values
# X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
clf1 = GaussianNB()
clf1.fit(X_train, y_train)
print(clf1.score(X_test, y_test))
predictions1 = clf1.predict(X_test)

In [ ]:
pprint.pprint(metrics.classification_report(predictions1, y_test))

In [ ]:
clf2 = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)
clf2 = clf2.fit(X_train, y_train)
predictions2 = clf2.predict(X_test)
pprint.pprint(metrics.classification_report(y_test, predictions2))

In [ ]:
print(clf2.score(X_test, y_test))


In [ ]:
# %load tpot_mnist_pipeline.py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:0.5522662309923989
exported_pipeline = GaussianNB()

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [ ]:
X = final.iloc[:10000][['home_team','away_team','EH','EA']]#.values
y = final.iloc[:10000]['over_under_2.5']#.values

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:

# Add noisy features
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape

# Create a simple classifier
classifier = svm.LinearSVC(random_state=random_state)
classifier.fit(X_train, y_train)
y_score = classifier.decision_function(X_test)

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_score)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=0)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 0.2])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
from itertools import cycle

from sklearn.metrics import roc_curve, auc

from scipy import interp

In [ ]:
from sklearn.preprocessing import label_binarize

In [ ]:
# Binarize the output
y = label_binarize(y, classes=[0, 1])
n_classes = y.shape[1]

# Add noisy features to make the problem harder

y_score = clf.decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
aa = pd.DataFrame(list(zip(predictions,y_test)))

In [ ]:
a = pd.DataFrame(y_train)
a['over_under_2.5'].value_counts(normalize=True).sort_index()

In [ ]:
ys = aa[aa[0]!=0].values

In [ ]:
yp = ys[:,0]
yt = ys[:,1]

In [ ]:
pprint.pprint(metrics.classification_report(yp, yt))

In [ ]:
# %load tpot_mnist_pipeline.py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:0.5523999202370016
exported_pipeline = GaussianNB()

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [ ]:
X = final.iloc[:][['home_team','away_team','EH','EA']].values
y = final.iloc[:]['over_under_2.5'].values
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.75, test_size=0.25, random_state=42)

In [ ]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)

gbm.fit(X_train, y_train)

predictions = gbm.predict(X_test)

In [ ]:
pprint.pprint(metrics.classification_report(predictions, y_test))

In [ ]:
ac = metrics.accuracy_score(predictions, y_test)

In [ ]:
print(ac)

In [ ]:
metrics.confusion_matrix(predictions, y_test)

In [ ]:

conf_arr = metrics.confusion_matrix(predictions, y_test)

norm_conf = []
for i in conf_arr:
    a = 0
    tmp_arr = []
    a = sum(i, 0)
    for j in i:
        tmp_arr.append(float(j)/float(a))
    norm_conf.append(tmp_arr)

fig = plt.figure()
plt.clf()
ax = fig.add_subplot(111)
ax.set_aspect(1)
res = ax.imshow(np.array(norm_conf), cmap=plt.cm.jet, 
                interpolation='nearest')

width, height = conf_arr.shape

for x in range(width):
    for y in range(height):
        ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                    horizontalalignment='center',
                    verticalalignment='center')

cb = fig.colorbar(res)
alphabet1 = ['Predicted-O','Predicted-U']
alphabet2 = ['Actual-O','Actual-U']
plt.xticks(range(width), alphabet2[:width])
plt.yticks(range(height), alphabet1[:height])
plt.savefig('confusion_matrix.png', format='png')

### sources

In [ ]:
#https://stackoverflow.com/questions/5821125/how-to-plot-confusion-matrix-with-string-axis-rather-than-integer-in-python

In [ ]:
# sudo apt-get install build-essentials gfortran gcc